# 📘 LightGBM Binary Classification Pipeline

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor
import matplotlib.pyplot as plt


In [8]:
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
sample_submission = pd.read_csv('sample_submission.csv')
train_sorted = train.sort_values(proxy_col).reset_index(drop=True)
test_sorted = test.sort_values(proxy_col).reset_index(drop=True)


In [9]:

for lag in [1, 2, 3, 5, 10]:
    train_sorted[f'label_lag{lag}'] = train_sorted['label'].shift(lag).fillna(0)
    train_sorted[f'{proxy_col}_lag{lag}'] = train_sorted[proxy_col].shift(lag).fillna(0)
    test_sorted[f'{proxy_col}_lag{lag}'] = test_sorted[proxy_col].shift(lag).fillna(0)
    # For label_lag, only fill test with zero (unless doing iterative prediction hack)
    test_sorted[f'label_lag{lag}'] = 0

# Drop any non-feature columns
drop_cols = ['label', 'ID', 'id']
X_train = train_sorted.drop(drop_cols, axis=1, errors='ignore')
y_train = train_sorted['label']
X_test = test_sorted.drop(drop_cols, axis=1, errors='ignore')

In [10]:
X_test = X_test[X_train.columns]

# KFold as before
from sklearn.model_selection import KFold
import numpy as np
from xgboost import XGBRegressor

kf = KFold(n_splits=5)
oof = np.zeros(len(X_train))
test_preds = np.zeros(len(X_test))

for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    model = XGBRegressor(n_estimators=200, max_depth=6)
    model.fit(X_tr, y_tr)
    oof[val_idx] = model.predict(X_val)
    test_preds += model.predict(X_test) / kf.n_splits

# Submission
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['prediction'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
print('Submission saved!')

Submission saved!


In [11]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['prediction'] = test_preds
sample_submission.to_csv('submission.csv', index=False)

from scipy.stats import pearsonr
score = pearsonr(train_sorted['label'], oof)[0]
print("OOF Score:", score)


OOF Score: 0.11703450904692425
